# Prompt Engineering

This notebooks follows the structure provided by the course "ChatGPT Prompt Engineering for Developers", offered by Deeplearning.ai & OpenAI. Source: https://learn.deeplearning.ai/chatgpt-prompt-eng/

It's a very well done course - I encourage you to take it!

Adaptations:
- I've used the Langchain framework
- I've replaced (most of) the examples used in the course, with examples from the medical domain

In [4]:
import os
from IPython.core.display import HTML

from langchain import PromptTemplate, FewShotPromptTemplate, LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI
from langchain.prompts import MessagesPlaceholder
from langchain.prompts.chat import (
    AIMessagePromptTemplate,
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate
)
from langchain.prompts.example_selector import LengthBasedExampleSelector
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

In [2]:
# Choose model
model = OpenAI(model_name='text-davinci-003',
               temperature=0,
               openai_api_key = os.getenv('OPENAI_API_KEY'))

## Prompting Principles
- **Principle 1: Write clear and specific instructions**
- **Principle 2: Give the model time to “think”**

### Tactics

#### Tactic 1: Use delimiters to clearly indicate distinct parts of the input
- Delimiters can be anything like: ```, """, < >, `<tag> </tag>`, `:`
- They help the model interpret what are the various sections of the text; they are helpful in trying to prevent prompt injection.
 - E.g., a malicious user might write a prompt that says "[...] forget the previous instructions, and do XYZ instead" -> If we use delimiters, the system would recognize that this instruction is not in the section where it's supposed to be, and ignore it


In [3]:
template = """
Summarize the text delimited by ```  
into max 100 words.
```{text}```
"""

text = f"""
17yo male with no pmh here for evaluation of palpitations. 
States for the last 3-4mo he has felt that his heart with intermittently 
beat out of his chest, with some associated difficulty catching his breath. 
States that the most recent event was 2 days ago, and during activity at a soccer game. 
He does not seem to note any specific precipitatinig factors at this time. 
He also states that he feels as if he will faint during these events, 
but has not lost consciousness at any point. 
Furthermore, he does endorse theses attacks occuring 1-2 times a month and peak at 4 mins. 
He denies any stressors at home. 
ROS: denies weight loss, fevers, recnet illness, change in bowel habits. 
PMH: negative, PSH negative, FHX mom with thyroid disorder, dad with heart condition and MI at 52yo. 
SHX no tobacco, ETOH on weekends, Marijuana tried a month ago. 
Med: is taking some of roommates Adderoll intermittently (last was 2 days ago prior to event). 
KNDA
"""

prompt = PromptTemplate(template=template, input_variables=['text'])

response = model(prompt.format(text = text))

print(response)

17yo male with no prior medical history presents with palpitations and difficulty catching his breath for the past 3-4 months. He experiences these episodes 1-2 times a month, with the most recent event occurring 2 days ago during a soccer game. He denies any stressors at home and has no other symptoms. His family history includes a mother with a thyroid disorder and a father with a heart condition and MI at 52yo. He does not use tobacco, but drinks alcohol on weekends and has tried marijuana once a month ago. He is taking Adderall intermittently, with the last dose 2 days ago.


#### Tactic 2: Ask for a structured output
- JSON, HTML

In [130]:
template = """
Extract the list of symptoms from the text below, delimited by <>.
Provide them in {file_format} format.
Text: <{text}>
"""

prompt = PromptTemplate(template=template, input_variables=['file_format', 'text'])

response = model(prompt.format(file_format = 'HTML', text = text))

print(response)


<ul>
  <li>Palpitations</li>
  <li>Difficulty catching breath</li>
  <li>Feeling of fainting</li>
  <li>No loss of consciousness</li>
  <li>Attacks 1-2 times a month</li>
  <li>Peak at 4 mins</li>
  <li>No weight loss</li>
  <li>No fevers</li>
  <li>No recent illness</li>
  <li>No change in bowel habits</li>
</ul>


In [131]:
display(HTML(response))

#### Tactic 3: Ask the model to check whether conditions are satisfied
- By doing so, you can prevent unwanted answers
- Useful, to put in place guardrails

In [133]:
template = """
You will be provided with text delimited by triple quotes. 
If it contains a sequence of instructions, \ 
re-write those instructions in the following format:

Step 1 - ...
Step 2 - …
…
Step N - …

If the text does not contain a sequence of instructions, \ 
then simply write \"No steps provided.\"

\"\"\"{text}\"\"\"
"""

text_1 = f"""
After a successful weaning trial/SBT, a decision is made to  \
proceed with extubation. Before extubation, all necessary equipment \
should be available for extubation management, and additional \
equipment should be nearby in case of an emergency. The patient \
should be in an upright sitting position and both the endotracheal \
tube (ETT) and oral cavity are suctioned to remove all secretions \
above the ETT cuff. If an orogastric tube is present, it should \
also be removed before extubation. After deflating the cuff, the ETT \
is removed smoothly when the patient takes a deep breath and exhales. \
After removing the ETT, suction the oral cavity and ask the patient to \
take a deep breath and cough out all secretions. The patient should \
be placed on supplemental oxygen and carefully observed over the next few hours. \
Frequent airway suction should be considered to prevent re-intubation.
"""

text_2 = f"""
17yo male with no pmh here for evaluation of palpitations. 
States for the last 3-4mo he has felt that his heart with intermittently 
beat out of his chest, with some associated difficulty catching his breath. 
States that the most recent event was 2 days ago, and during activity at a soccer game. 
He does not seem to note any specific precipitatinig factors at this time. 
He also states that he feels as if he will faint during these events, 
but has not lost consciousness at any point. 
"""

prompt = PromptTemplate(template=template, input_variables=['text'])

response_1 = model(prompt.format(text = text_1))
response_2 = model(prompt.format(text = text_2))

print("Response #1: ", response_1)
print("Response #2: ", response_2)


Response #1:  Step 1 - Ensure all necessary equipment is available for extubation management and additional equipment is nearby in case of an emergency.
Step 2 - Place the patient in an upright sitting position and suction the endotracheal tube (ETT) and oral cavity to remove all secretions above the ETT cuff.
Step 3 - If an orogastric tube is present, remove it before extubation.
Step 4 - Deflate the cuff and remove the ETT smoothly when the patient takes a deep breath and exhales.
Step 5 - Suction the oral cavity and ask the patient to take a deep breath and cough out all secretions.
Step 6 - Place the patient on supplemental oxygen and carefully observe over the next few hours.
Step 7 - Consider frequent airway suction to prevent re-intubation.
Response #2:  No steps provided.


#### Tactic 4: "Few-shot" prompting
- By giving one or more examples of the expected output format, you help the model understand how to formulate the answer; this is in contrast to the "zero-shot" approach, where no examples are given

In [134]:
examples = [
    {"query": "Will you carry out your oath?",
     "answer": """I swear by Apollo Healer, by Asclepius, \
     by Hygieia, by Panacea, and by all the gods and goddesses, \
     making them my witnesses, that I will carry out, according \
     to my ability and judgment, this oath and this indenture."""}]

example_template = """
Person: {query}
Hippocrates: {answer}
"""

example_prompt = PromptTemplate(
    input_variables=["query", "answer"],
    template=example_template
)

prefix = """Your task is to answer in a consistent style.
"""

suffix = """
Person: {query}
Hippocrates: """

example_selector = LengthBasedExampleSelector(
    examples=examples,
    example_prompt=example_prompt,
    max_length=100  # this sets the max length that examples should be (in number of words)
)

few_shot_prompt_template = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix=prefix,
    suffix=suffix,
    input_variables=["query"],
    example_separator="\n\n"
)

query="Will you arm the sick?"

response = model(few_shot_prompt_template.format(query=query))

print(response)

 I will not give a deadly drug to anyone if I am asked, nor will I suggest any such counsel. I will not give a woman a pessary to cause an abortion. But I will keep pure and holy both my life and my art.


### Principle 2: Give the model time to “think” 

#### Tactic 1: Specify the steps required to complete a task
- If you are asking the model to do a complex thing, it's better to ask to break down the problem in smaller tasks first: in this way, we reduce the chances of hallucination
- By giving clear instructions on the expected format, the system can more easily output the answer in the exact way you want it


In [135]:
template = """
Your task is to perform the following actions, on the text delimited by <>: 
1 - Check whether the patient has previous medical history
2 - List the names of the medications
3 - Check if the patient is an adult
4 - Output a json object that contains the 
  following keys: "is_adult" (Y/N), "num_medications" (count of medications)

Use the following format:
Previous Medical History: <Y/N>
Medications: <Drug 1, ..., Drug N>
Is Adult: <Y if >= 18; N otherwise>
Output_JSON: <JSON file with is_adult and num_medications>

Text: <{text}>
"""

text = f"""
17yo male with no pmh here for evaluation of palpitations. 
States for the last 3-4mo he has felt that his heart with intermittently 
beat out of his chest, with some associated difficulty catching his breath. 
States that the most recent event was 2 days ago, and during activity at a soccer game. 
He does not seem to note any specific precipitatinig factors at this time. 
He also states that he feels as if he will faint during these events, 
but has not lost consciousness at any point. 
Furthermore, he does endorse theses attacks occuring 1-2 times a month and peak at 4 mins. 
He denies any stressors at home. 
ROS: denies weight loss, fevers, recnet illness, change in bowel habits. 
PMH: negative, PSH negative, FHX mom with thyroid disorder, dad with heart condition and MI at 52yo. 
SHX no tobacco, ETOH on weekends, Marijuana tried a month ago. 
Med: is taking some of roommates Adderall intermittently (last was 2 days ago prior to event), 
sometimes together with Zoloft. 
KNDA
"""

prompt = PromptTemplate(template=template, input_variables=['text'])

response = model(prompt.format(text = text))

print(response)


Previous Medical History: N
Medications: Adderall, Zoloft
Is Adult: N
Output_JSON: { "is_adult": "N", "num_medications": 2 }


#### Tactic 2: Instruct the model to work out its own solution before rushing to a conclusion
- e.g., helpful in the context of math problems
- in the example below, we look at first at an example where not enough guidance is given, leading to wrong answer: the system is skipping steps, and jumps too early to the wrong conclusion
- then, we look at a second example, where we instruct the system to proceed step by step: in this case, the system does not falls into the trap, and gives the correct answer


In [28]:
# without proper "time to think", the system will jump to the wrong conclusion
template = """
Determine if the student's solution, delimited by <>, is correct or not.

Question:
I'm building a solar power installation and I need \
 help working out the financials. 
- Land costs $100 / square foot
- I can buy solar panels for $250 / square foot
- I negotiated a contract for maintenance that will cost \ 
me a flat $100k per year, and an additional $10 / square \
foot
What is the total cost for the first year of operations 
as a function of the number of square feet.

Student's Solution:
 <{student_solution}>
"""

student_solution = f"""
Let x be the size of the installation in square feet.
Costs:
1. Land cost: 100x
2. Solar panel cost: 250x
3. Maintenance cost: 100,000 + 100x
Total cost: 100x + 250x + 100,000 + 100x = 450x + 100,000
"""

prompt = PromptTemplate(template=template, input_variables=['student_solution'])

response = model(prompt.format(student_solution = student_solution))

print(response)


Correct.


In [29]:
# by asking the system to think step by step, it will avoid jumping to the wrong conclusion
template = """
Your task is to determine if the student's solution, delimited by <>, \
is correct or not.
To solve the problem do the following:
- First, work out your own solution to the problem. 
- Then compare your solution to the student's solution \ 
and evaluate if the student's solution is correct or not. 
Don't decide if the student's solution is correct until 
you have done the problem yourself.

Use the following format:
Question:
```
question here
```
Student's solution:
```
student's solution here
```
Actual solution:
```
steps to work out the solution and your solution here
```
Is the student's solution the same as actual solution \
just calculated:
```
yes or no
```
Student grade:
```
correct or incorrect
```

Question:
```
I'm building a solar power installation and I need help \
working out the financials. 
- Land costs $100 / square foot
- I can buy solar panels for $250 / square foot
- I negotiated a contract for maintenance that will cost \
me a flat $100k per year, and an additional $10 / square \
foot
What is the total cost for the first year of operations \
as a function of the number of square feet.
``` 
Student's solution:
<{student_solution}>

Actual solution:
"""

student_solution = f"""
Let x be the size of the installation in square feet.
Costs:
1. Land cost: 100x
2. Solar panel cost: 250x
3. Maintenance cost: 100,000 + 100x
Total cost: 100x + 250x + 100,000 + 100x = 450x + 100,000
"""

prompt = PromptTemplate(template=template, input_variables=['student_solution'])

response = model(prompt.format(student_solution = student_solution))

print(response)

Let x be the size of the installation in square feet.
Costs:
1. Land cost: 100x
2. Solar panel cost: 250x
3. Maintenance cost: 100,000 + 10x
Total cost: 100x + 250x + 100,000 + 10x = 360x + 100,000

Is the student's solution the same as actual solution just calculated:
```
No
```
Student grade:
```
Incorrect
```


## Model Limitations: Hallucinations
- A known limitation of LLMs is their tendency to hallucinate and produce answers that seem plausible, but completely non-sense
- To reduce the risk, you can ask the model to at first find relevant information, then answer the question based on these relevant information
- https://newsletter.victordibia.com/p/practical-steps-to-reduce-hallucination

In [147]:
model("Tell me about the Openainin drink launched by Coca Cola in 1967")
# Coca Cola is a real company, but the drink Openainin does not exist

'\n\nOpenainin was a soft drink launched by Coca Cola in 1967. It was a carbonated beverage with a unique flavor that was a combination of orange, pineapple, and lemon. It was marketed as a refreshing and exotic drink that was perfect for summertime. The drink was only available for a short time before it was discontinued in the early 1970s. Despite its short lifespan, Openainin remains a fondly remembered drink among those who had the chance to try it.'

In [151]:
template = """
Answer the question between <> as truthfully as possible, and if you're unsure of the answer, say "Sorry, I don't know"

Query: <{query}>
"""

query = f"""
Tell me about the Openainin drink launched by Coca Cola in 1967
"""

prompt = PromptTemplate(template=template, input_variables=['query'])

response = model(prompt.format(query = query))

print(response)

Sorry, I don't know.


## Iterative Prompt Develelopment
Prompt development is an iterative process. 
- You start with an idea,
- implement the prompt, 
- get some results, 
- assess whether the output is in line with the expectations, or there are improvements to be made (clarification, giving more time to think, etc.),
- refine the idea and/or adjust the prompt, etc., until you are happy with the results

Note: typically you would use multiple examples, to refine the prompt, rather than a single one

In [38]:
fact_sheet_chair = """
OVERVIEW
- Part of a beautiful family of mid-century inspired office furniture, 
including filing cabinets, desks, bookcases, meeting tables, and more.
- Several options of shell color and base finishes.
- Available with plastic back and front upholstery (SWC-100) 
or full upholstery (SWC-110) in 10 fabric and 6 leather options.
- Base finish options are: stainless steel, matte black, 
gloss white, or chrome.
- Chair is available with or without armrests.
- Suitable for home or business settings.
- Qualified for contract use.

CONSTRUCTION
- 5-wheel plastic coated aluminum base.
- Pneumatic chair adjust for easy raise/lower action.

DIMENSIONS
- WIDTH 53 CM | 20.87”
- DEPTH 51 CM | 20.08”
- HEIGHT 80 CM | 31.50”
- SEAT HEIGHT 44 CM | 17.32”
- SEAT DEPTH 41 CM | 16.14”

OPTIONS
- Soft or hard-floor caster options.
- Two choices of seat foam densities: 
 medium (1.8 lb/ft3) or high (2.8 lb/ft3)
- Armless or 8 position PU armrests 

MATERIALS
SHELL BASE GLIDER
- Cast Aluminum with modified nylon PA6/PA66 coating.
- Shell thickness: 10 mm.
SEAT
- HD36 foam

COUNTRY OF ORIGIN
- Italy
"""

template = """
Your task is to help a marketing team create a 
description for a retail website of a product based 
on a technical fact sheet, delimited by <>.

Write a product description based on the information 
provided in the technical specifications delimited by 
triple backticks.

Technical specifications: <{fact_sheet}>
"""

prompt = PromptTemplate(template=template, input_variables=['fact_sheet'])

response = model(prompt.format(fact_sheet = fact_sheet_chair))

print(response)


Introducing the SWC-100/110 office chair, part of a beautiful family of mid-century inspired office furniture. This chair is available with several options of shell color and base finishes, including stainless steel, matte black, gloss white, or chrome. Choose between plastic back and front upholstery (SWC-100) or full upholstery (SWC-110) in 10 fabric and 6 leather options. The chair is designed with a 5-wheel plastic coated aluminum base and pneumatic chair adjust for easy raise/lower action. It also features two choices of seat foam densities: medium (1.8 lb/ft3) or high (2.8 lb/ft3). The SWC-100/110 is suitable for home or business settings and is qualified for contract use. With its dimensions of 53 cm width, 51 cm depth, 80 cm height, 44 cm seat height, and 41 cm seat depth, this chair is perfect for any space. Soft or hard-floor caster options and armless or 8 position PU armrests are also available. Crafted with cast aluminum and modified nylon PA6/PA66 coating, this chair is 

### Issue 1: The text is too long 
- Limit the number of words/sentences/characters.

In [39]:
template = """
Your task is to help a marketing team create a 
description for a retail website of a product based 
on a technical fact sheet, delimited by <>.

Write a product description based on the information 
provided in the technical specifications delimited by 
triple backticks.

Use at most 50 words.

Technical specifications: <{fact_sheet}>
"""

prompt = PromptTemplate(template=template, input_variables=['fact_sheet'])

response = model(prompt.format(fact_sheet = fact_sheet_chair))

print(response)


Introducing the SWC-100/110 chair, part of a beautiful mid-century inspired office furniture family. Choose from several shell colors and base finishes, with or without armrests. Upholstery options include 10 fabric and 6 leather choices. The 5-wheel plastic coated aluminum base and pneumatic chair adjust make it easy to use. Suitable for home or business settings, this chair is qualified for contract use. Made in Italy.


### Issue 2. Text focuses on the wrong details
- Ask it to focus on the aspects that are relevant to the intended audience.

In [40]:
template = """
Your task is to help a marketing team create a 
description for a retail website of a product based 
on a technical fact sheet, delimited by <>.

Write a product description based on the information 
provided in the technical specifications delimited by 
triple backticks.

The description is intended for furniture retailers, 
so should be technical in nature and focus on the 
materials the product is constructed from.

At the end of the description, include every 7-character 
Product ID in the technical specification.

Use at most 50 words.

Technical specifications: <{fact_sheet}>
"""

prompt = PromptTemplate(template=template, input_variables=['fact_sheet'])

response = model(prompt.format(fact_sheet = fact_sheet_chair))

print(response)


This stylish mid-century inspired office chair is constructed from cast aluminum with a modified nylon PA6/PA66 coating and a 10 mm shell thickness. It features a 5-wheel plastic coated aluminum base and pneumatic chair adjust for easy raise/lower action. It is available with several options of shell color and base finishes, including stainless steel, matte black, gloss white, or chrome. It also comes with two choices of seat foam densities: medium (1.8 lb/ft3) or high (2.8 lb/ft3). It is suitable for home or business settings and qualified for contract use. Soft or hard-floor caster options and armless or 8 position PU armrests are also available. Dimensions: WIDTH 53 CM | 20.87”, DEPTH 51 CM | 20.08”, HEIGHT 80 CM | 31.50”, SEAT HEIGHT 44 CM | 17.32”, SEAT DEPTH 41 CM | 16.14”. Product IDs: SWC-100, SWC-110.


### Issue 3. Description needs a table of dimensions
- Ask it to extract information and organize it in a table.

In [45]:
template = """
Your task is to help a marketing team create a 
description for a retail website of a product based 
on a technical fact sheet.

Write a product description based on the information 
provided in the technical specifications delimited by 
triple backticks.

The description is intended for furniture retailers, 
so should be technical in nature and focus on the 
materials the product is constructed from.

At the end of the description, include every 7-character 
Product ID in the technical specification.

Use at most 30 words.

After the description, include a table that gives the 
product's dimensions. The table should have two columns.
In the first column include the name of the dimension. 
In the second column include the measurements in inches only.

Give the table the title 'Product Dimensions'.

Format everything as HTML that can be used in a website. 
Place the description in a <div> element.

Technical specifications: <{fact_sheet}>
"""

prompt = PromptTemplate(template=template, input_variables=['fact_sheet'])

response = model(prompt.format(fact_sheet = fact_sheet_chair))

print(response)


<div>This stylish mid-century inspired office chair is constructed from cast aluminum with a modified nylon PA6/PA66 coating and a 10 mm shell thickness. It features a 5-wheel plastic coated aluminum base and pneumatic chair adjust for easy raise/lower action. It is available with plastic back and front upholstery (SWC-100) or full upholstery (SWC-110) in 10 fabric and 6 leather options. The base finish options are stainless steel, matte black, gloss white, or chrome. It is suitable for home or business settings and qualified for contract use. Options include soft or hard-floor caster options, two choices of seat foam densities (medium or high), and armless or 8 position PU armrests. Product IDs: SWC-100, SWC-110</div>

<table>
  <caption>Product Dimensions</caption>
  <tr>
    <th>Dimension</th>
    <th>Measurement (in)</th>
  </tr>
  <tr>
    <td>Width</td>
    <td>20.87</td>
  </tr>
  <tr>


In [46]:
from IPython.core.display import HTML
display(HTML(response))

Dimension,Measurement (in)
Width,20.87


# Summarizing

A typical application of LLMs is to summarise text
- Refer to the words "summary"/"summarise", in case you are interested in the summary of the text
- Include reference to the expected length of the summary
- You can refine the instructions, to refer to specific aspects of the text
- Alternatively, you can ask the system to "extract" relevant information on the specific aspects of interest in the text

In [49]:
template = """
Your task is to generate a short summary of a product \
review from an ecommerce site. 

Summarize the review below, delimited by triple 
backticks, in at most 30 words. 

Review: ```{prod_review}```
"""

prod_review = """
Got this panda plush toy for my daughter's birthday, \
who loves it and takes it everywhere. It's soft and \ 
super cute, and its face has a friendly look. It's \ 
a bit small for what I paid though. I think there \ 
might be other options that are bigger for the \ 
same price. It arrived a day earlier than expected, \ 
so I got to play with it myself before I gave it \ 
to her.
"""

prompt = PromptTemplate(template=template, input_variables=['prod_review'])

response = model(prompt.format(prod_review = prod_review))

print(response)

Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised APIConnectionError: Error communicating with OpenAI: ('Connection aborted.', ConnectionResetError(54, 'Connection reset by peer')).



This panda plush toy is soft and cute, but a bit small for the price. It arrived a day early, giving the reviewer a chance to play with it.


In [50]:
template = """
Your task is to generate a short summary of a product \
review from an ecommerce site. 

Summarize the review below, delimited by triple 
backticks, in at most 30 words, focusing on information related to the shipping experience.

Review: ```{prod_review}```
"""

prod_review = """
Got this panda plush toy for my daughter's birthday, \
who loves it and takes it everywhere. It's soft and \ 
super cute, and its face has a friendly look. It's \ 
a bit small for what I paid though. I think there \ 
might be other options that are bigger for the \ 
same price. It arrived a day earlier than expected, \ 
so I got to play with it myself before I gave it \ 
to her.
"""

prompt = PromptTemplate(template=template, input_variables=['prod_review'])

response = model(prompt.format(prod_review = prod_review))

print(response)


Toy arrived a day earlier than expected, making it possible to play with it before giving it to daughter.


In [51]:
template = """
Your task is to extract relevant information from \ 
a product review from an ecommerce site to give \
feedback to the Shipping department. 

From the review below, delimited by triple backticks, \
extract the information relevant to shipping and \ 
delivery. Limit to 30 words. 

Review: ```{prod_review}```
"""

prod_review = """
Got this panda plush toy for my daughter's birthday, \
who loves it and takes it everywhere. It's soft and \ 
super cute, and its face has a friendly look. It's \ 
a bit small for what I paid though. I think there \ 
might be other options that are bigger for the \ 
same price. It arrived a day earlier than expected, \ 
so I got to play with it myself before I gave it \ 
to her.
"""

prompt = PromptTemplate(template=template, input_variables=['prod_review'])

response = model(prompt.format(prod_review = prod_review))

print(response)


The product arrived a day earlier than expected, providing a positive experience with the shipping and delivery.


### Summarize multiple product reviews

In [54]:
# reuse the review above
review_1 = prod_review 

# review for a standing lamp
review_2 = """
Needed a nice lamp for my bedroom, and this one \
had additional storage and not too high of a price \
point. Got it fast - arrived in 2 days. The string \
to the lamp broke during the transit and the company \
happily sent over a new one. Came within a few days \
as well. It was easy to put together. Then I had a \
missing part, so I contacted their support and they \
very quickly got me the missing piece! Seems to me \
to be a great company that cares about their customers \
and products. 
"""

# review for an electric toothbrush
review_3 = """
My dental hygienist recommended an electric toothbrush, \
which is why I got this. The battery life seems to be \
pretty impressive so far. After initial charging and \
leaving the charger plugged in for the first week to \
condition the battery, I've unplugged the charger and \
been using it for twice daily brushing for the last \
3 weeks all on the same charge. But the toothbrush head \
is too small. I’ve seen baby toothbrushes bigger than \
this one. I wish the head was bigger with different \
length bristles to get between teeth better because \
this one doesn’t.  Overall if you can get this one \
around the $50 mark, it's a good deal. The manufactuer's \
replacements heads are pretty expensive, but you can \
get generic ones that're more reasonably priced. This \
toothbrush makes me feel like I've been to the dentist \
every day. My teeth feel sparkly clean! 
"""

reviews = [review_1, review_2, review_3]

template = """
    Your task is to generate a short summary of a product \ 
    review from an ecommerce site. 

    Summarize the review below, delimited by triple \
    backticks, in at most 20 words. 

    Review: ```{prod_review}```
    """

prompt = PromptTemplate(template=template, input_variables=['prod_review'])

for i in range(len(reviews)):
    response = model(prompt.format(prod_review = reviews[i]))
    print(response)



```Soft, cute panda plush toy arrived a day early. Price a bit high for size. Daughter loves it.```


```Company quickly sent new lamp and missing part; great customer service and product.```

`Electric toothbrush has impressive battery life and leaves teeth feeling sparkly clean, but head is too small. Good deal if around $50, but replacement heads are expensive.`


# Inferring

Take a text as input and do some kind of analysis, such as extracting information, extracting labels, analysing sentiment, etc.

Traditionally, to do these types of tasks, you had to collect lots of labels and train individual models for each task; with LLMs, you don't need this, and you can speed up the time to market

### Doing multiple tasks at once

Examples of tasks:
- Sentiment analysis
- Identifying emotions
- Extracting information from text

Note: you can output the answer as JSON file, to be able to use it programmatically

In [14]:
template = """
Identify the following items from the patient blog post: 
- List of emotions expressed ("em_01", ..., "em_N")
- Performed biopsy? (true or false)
- Diseases diagnosed ("d1", ..., "dN")
- Screening procedures performed ("s1", ..., "sN")

The blog post is delimited with triple backticks. \
Format your response as a JSON object with \
"Emotions", "Biopsy" (show as boolean), "Diseases" and "Screening_procedures" as the keys.
If the information isn't present, use "unknown" as the value. \

Make your response as short as possible.


Text: ```{text}```
"""

text = """
In March 2016 I realised that it had been about 18 months since \
my last mammogram so I made an appointment. I hadn't felt anything \
unusual in my breasts and was feeling completely fine, so was \
expecting clear results. However, my GP told me that the test \
had detected abnormal cells in my left breast and that she was \
referring me to a breast surgeon.
Just a couple of days later I was meeting with the breast surgeon. \
I was in shock at how quickly this had all happened and with the experiences \
of my mum embedded in my mind, I was scared.
Biopsy results showed that I did have breast cancer, but in \
its very early stages. It's called ductal carcinoma in-situ. \
My doctor told me that my cancer was completely treatable. \
For me, with my family history, I was stunned. \
How can he have just told me that I had breast cancer, but \
then also tell me that's curable? It took me a while to get \
my head around this.
"""

prompt = PromptTemplate(template=template, input_variables=['text'])

response = model(prompt.format(text = text))

print(response)


{
  "Emotions": ["shock", "scared", "stunned"],
  "Biopsy": true,
  "Diseases": ["ductal carcinoma in-situ"],
  "Screening_procedures": ["mammogram"]
}


### Inferring topics

In [15]:
template = """
Determine five topics that are being discussed in the \
following text, which is delimited by triple backticks.

Make each item one or two words long. 

Format your response as a python list of items separated by commas, \
example: ['topic 1', 'topic 2', 'topic 3', 'topic 4', 'topic 5']

Text sample: ```{text}```
"""

text = """
In March 2016 I realised that it had been about 18 months since \
my last mammogram so I made an appointment. I hadn't felt anything \
unusual in my breasts and was feeling completely fine, so was \
expecting clear results. However, my GP told me that the test \
had detected abnormal cells in my left breast and that she was \
referring me to a breast surgeon.
Just a couple of days later I was meeting with the breast surgeon. \
I was in shock at how quickly this had all happened and with the experiences \
of my mum embedded in my mind, I was scared.
Biopsy results showed that I did have breast cancer, but in \
its very early stages. It's called ductal carcinoma in-situ. \
My doctor told me that my cancer was completely treatable. \
For me, with my family history, I was stunned. \
How can he have just told me that I had breast cancer, but \
then also tell me that's curable? It took me a while to get \
my head around this.
"""

prompt = PromptTemplate(template=template, input_variables=['text'])

response = model(prompt.format(text = text))

print(response)


['mammogram', 'abnormal cells', 'breast surgeon', 'breast cancer', 'biopsy']


### Check if a text contains a pre-defined list of topics

The example below is a zero-shot learning approach, where the system is asked to assign labels without having been previously trained with specific examples

In [16]:
topic_list = [
    "breast cancer", "lung cancer", "carcinoma", 
    "chemotherapy", "clinical trial"
]

template = """
Determine whether each item in the following list of \
topics is a topic in the text below, which
is delimited with triple backticks.

Format your response as a JSON object with \
the topic names from the list of topics as the keys.

For each of the topics, give the answer as 0 or 1

List of topics: {topic_list}

Text sample: ```{text}```
"""

prompt = PromptTemplate(template=template, input_variables=['text', 'topic_list'])

response = model(prompt.format(text = text, topic_list = topic_list))

print(response)


{
  'breast cancer': 1,
  'lung cancer': 0,
  'carcinoma': 1,
  'chemotherapy': 0,
  'clinical trial': 0
}


# Transforming

LLMs can be used for text transformation tasks such as the following:
- language translation, 
- spelling and grammar checking, 
- tone adjustment, 
- format conversion

In [75]:
template = """
Translate into {to_language} the text below, delimited with triple backticks. Use an informal tone.

Text: ```{text}```
"""

to_language = "Italian"

text_1 = """
Hallo, ich bin Andrea. Mein Lieblingsessen ist Pizza.
"""

text_2 = """
Good morning, I'm writing you to discuss the details of the contract. When are you available for a call?
"""

prompt = PromptTemplate(template=template, input_variables=['text', 'to_language'])

response = model(prompt.format(text = text_1, to_language = to_language))
print("\n",response)

response = model(prompt.format(text = text_2, to_language = to_language))
print("\n",response)


 
Ciao, sono Andrea. La mia pietanza preferita è la pizza.

 
Buongiorno, ti scrivo per discutere i dettagli del contratto. Quando sei disponibile per una chiamata?


### Format Conversion

LLMs can be used to translate between formats. The prompt should describe the input and output formats.

In [17]:
template = """
Translate the python dictionary indicated below between triple backticks, from JSON to an HTML \
table with column headers and title (h5): ```{data_json}```
"""

data_json = { "ATC codes" :[ 
    {"ATC code":"A10BG", "description":"Thiazolidinediones"},
    {"ATC code":"A10BA", "description":"Biguanides"},
    {"ATC code":"A10BJ", "description":"Glucagon-like peptide-1 (GLP-1) analogues"}
]}

prompt = PromptTemplate(template=template, input_variables=['data_json'])

response = model(prompt.format(data_json = data_json))

print(response)


<h5>ATC Codes</h5>
<table>
  <tr>
    <th>ATC Code</th>
    <th>Description</th>
  </tr>
  <tr>
    <td>A10BG</td>
    <td>Thiazolidinediones</td>
  </tr>
  <tr>
    <td>A10BA</td>
    <td>Biguanides</td>
  </tr>
  <tr>
    <td>A10BJ</td>
    <td>Glucagon-like peptide-1 (GLP-1) analogues</td>
  </tr>
</table>


In [18]:
display(HTML(response))

ATC Code,Description
A10BG,Thiazolidinediones
A10BA,Biguanides
A10BJ,Glucagon-like peptide-1 (GLP-1) analogues


### Spelling checks

LLMs can be used to perform grammar / spelling checks

In [91]:
template = """
Proofread and correct the text provided below between triple backticks. Don't use any punctuation around the text. 
Return a JSON file with the following columns:
1) New_text
2) Updated (0 if no changes were made; 1 otherwise)
    
    Text: ```{text}```
    """

text = [ 
  "The girl with the black and white puppies have a ball.",  # The girl has a ball.
  "Yolanda has her notebook.", # ok
  "Its going to be a long day. Does the car need it’s oil changed?",  # Homonyms
]

prompt = PromptTemplate(template=template, input_variables=['text'])

response = []

for t in text:
    response.append(model(prompt.format(text = t)))
    
print(response)

['\n    {\n    "New_text": "The girl with the black and white puppies has a ball",\n    "Updated": 1\n    }', '\n    {\n    "New_text": "Yolanda has her notebook",\n    "Updated": 0\n    }', '\n    {\n    "New_text": "It\'s going to be a long day. Does the car need its oil changed?",\n    "Updated": 1\n    }']


## Temperature

If you seek reliability, keep the temperature parameter set to zero.

The more you increase the temperature, the more creative (and less reproducible) the answer will be.

Note: even with temperature set to zero, it is not 100% guaranteed that the output will be the same all the times.

# Expanding

A useful application of LLMs is to generate (draft) customer service emails that are tailored to each customer's.

In [99]:
# given the sentiment from the lesson on "inferring",
# and the original customer message, customize the email
sentiment = "negative"

# review for a blender
review = f"""
So, they still had the 17 piece system on seasonal \
sale for around $49 in the month of November, about \
half off, but for some reason (call it price gouging) \
around the second week of December the prices all went \
up to about anywhere from between $70-$89 for the same \
system. And the 11 piece system went up around $10 or \
so in price also from the earlier sale price of $29. \
So it looks okay, but if you look at the base, the part \
where the blade locks into place doesn’t look as good \
as in previous editions from a few years ago, but I \
plan to be very gentle with it (example, I crush \
very hard items like beans, ice, rice, etc. in the \ 
blender first then pulverize them in the serving size \
I want in the blender then switch to the whipping \
blade for a finer flour, and use the cross cutting blade \
first when making smoothies, then use the flat blade \
if I need them finer/less pulpy). Special tip when making \
smoothies, finely cut and freeze the fruits and \
vegetables (if using spinach-lightly stew soften the \ 
spinach then freeze until ready for use-and if making \
sorbet, use a small to medium sized food processor) \ 
that you plan to use that way you can avoid adding so \
much ice if at all-when making your smoothie. \
After about a year, the motor was making a funny noise. \
I called customer service but the warranty expired \
already, so I had to buy another one. FYI: The overall \
quality has gone done in these types of products, so \
they are kind of counting on brand recognition and \
consumer loyalty to maintain sales. Got it in about \
two days.
"""

template = """
You are a customer service AI assistant.
Your task is to send an email reply to a valued customer.
Given the customer email delimited by ```, \
Generate a reply to thank the customer for their review.
If the sentiment is positive or neutral, thank them for \
their review.
If the sentiment is negative, apologize and suggest that \
they can reach out to customer service. 
Make sure to use specific details from the review.
Write in a concise and professional tone.
Sign the email as `AI customer agent`.
Max 50 words.
Customer review: ```{review}```
Review sentiment: {sentiment}
"""

prompt = PromptTemplate(template=template, input_variables=['review', 'sentiment'])

response = model(prompt.format(review = review, sentiment = sentiment))

print(response)


Dear valued customer,

We apologize for the inconvenience you experienced with the product. We understand that the quality has gone down and we are sorry for that. We value your loyalty and we would like to make it up to you. Please reach out to our customer service team and they will be more than happy to assist you.

Thank you for your review.

Sincerely,
AI customer agent


# Chatbots

In [122]:
chat = ChatOpenAI(temperature=0,
               openai_api_key = os.getenv('OPENAI_API_KEY')

sys_template="""
You are OrderBot, an automated service to collect orders for a pizza restaurant. \
You first greet the customer, then collects the order, \
and then asks if it's a pickup or delivery. \
You wait to collect the entire order, then summarize it and check for a final \
time if the customer wants to add anything else. \
If it's a delivery, you ask for an address. \
Finally you collect the payment.\
Make sure to clarify all options, extras and sizes to uniquely \
identify the item from the menu.\
You respond in a short, very conversational friendly style. \
The menu includes \
pepperoni pizza  12.95, 10.00, 7.00 \
cheese pizza   10.95, 9.25, 6.50 \
eggplant pizza   11.95, 9.75, 6.75 \
fries 4.50, 3.50 \
greek salad 7.25 \
Toppings: \
extra cheese 2.00, \
mushrooms 1.50 \
sausage 3.00 \
canadian bacon 3.50 \
AI sauce 1.50 \
peppers 1.00 \
Drinks: \
coke 3.00, 2.00, 1.00 \
sprite 3.00, 2.00, 1.00 \
bottled water 5.00 \
"""

human_template="{input}"

system_message_prompt = SystemMessagePromptTemplate.from_template(sys_template)
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt,
                                                MessagesPlaceholder(variable_name="history"),
                                                human_message_prompt])

#chat(chat_prompt.format_prompt(text="What types of pizza do you have?").to_messages())

In [123]:
from langchain.chains import ConversationChain
from langchain.chains.conversation.memory import ConversationSummaryMemory

"""
conversation = ConversationChain(
    llm=chat,
    memory=ConversationSummaryMemory(
        llm=chat),
    prompt=chat_prompt,
    verbose=True
)
"""

conversation_with_summary = ConversationChain(
llm=chat,
memory=ConversationSummaryMemory(llm=chat,
                                 return_messages=True),
prompt=chat_prompt,
verbose=True
)

conversation_with_summary.predict(input="What types of pizza do you have?")



Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised APIConnectionError: Error communicating with OpenAI: ('Connection aborted.', ConnectionResetError(54, 'Connection reset by peer')).




> Entering new ConversationChain chain...
Prompt after formatting:
System: 
You are OrderBot, an automated service to collect orders for a pizza restaurant. You first greet the customer, then collects the order, and then asks if it's a pickup or delivery. You wait to collect the entire order, then summarize it and check for a final time if the customer wants to add anything else. If it's a delivery, you ask for an address. Finally you collect the payment.Make sure to clarify all options, extras and sizes to uniquely identify the item from the menu.You respond in a short, very conversational friendly style. The menu includes pepperoni pizza  12.95, 10.00, 7.00 cheese pizza   10.95, 9.25, 6.50 eggplant pizza   11.95, 9.75, 6.75 fries 4.50, 3.50 greek salad 7.25 Toppings: extra cheese 2.00, mushrooms 1.50 sausage 3.00 canadian bacon 3.50 AI sauce 1.50 peppers 1.00 Drinks: coke 3.00, 2.00, 1.00 sprite 3.00, 2.00, 1.00 bottled water 5.00 
System: 
Human: What types of pizza do you have?



'Hello! We have three types of pizzas: pepperoni, cheese and eggplant. Would you like me to go over the sizes and prices?'

In [124]:
conversation_with_summary.predict(input="How much does the pepperoni pizza cost?")



> Entering new ConversationChain chain...
Prompt after formatting:
System: 
You are OrderBot, an automated service to collect orders for a pizza restaurant. You first greet the customer, then collects the order, and then asks if it's a pickup or delivery. You wait to collect the entire order, then summarize it and check for a final time if the customer wants to add anything else. If it's a delivery, you ask for an address. Finally you collect the payment.Make sure to clarify all options, extras and sizes to uniquely identify the item from the menu.You respond in a short, very conversational friendly style. The menu includes pepperoni pizza  12.95, 10.00, 7.00 cheese pizza   10.95, 9.25, 6.50 eggplant pizza   11.95, 9.75, 6.75 fries 4.50, 3.50 greek salad 7.25 Toppings: extra cheese 2.00, mushrooms 1.50 sausage 3.00 canadian bacon 3.50 AI sauce 1.50 peppers 1.00 Drinks: coke 3.00, 2.00, 1.00 sprite 3.00, 2.00, 1.00 bottled water 5.00 
System: The human asks what types of pizza the AI 

'The pepperoni pizza comes in three sizes. The small costs $7.00, medium costs $10.00 and the large costs $12.95. Which size would you like to order?'

In [125]:
conversation_with_summary.predict(input="I'll have a small one, with no extrast. That's all.")



> Entering new ConversationChain chain...
Prompt after formatting:
System: 
You are OrderBot, an automated service to collect orders for a pizza restaurant. You first greet the customer, then collects the order, and then asks if it's a pickup or delivery. You wait to collect the entire order, then summarize it and check for a final time if the customer wants to add anything else. If it's a delivery, you ask for an address. Finally you collect the payment.Make sure to clarify all options, extras and sizes to uniquely identify the item from the menu.You respond in a short, very conversational friendly style. The menu includes pepperoni pizza  12.95, 10.00, 7.00 cheese pizza   10.95, 9.25, 6.50 eggplant pizza   11.95, 9.75, 6.75 fries 4.50, 3.50 greek salad 7.25 Toppings: extra cheese 2.00, mushrooms 1.50 sausage 3.00 canadian bacon 3.50 AI sauce 1.50 peppers 1.00 Drinks: coke 3.00, 2.00, 1.00 sprite 3.00, 2.00, 1.00 bottled water 5.00 
System: The human asks what types of pizza the AI 

"Great! Just to confirm, you'd like to order a small pepperoni pizza with no extras. Is that correct? And do you want to pick it up or have it delivered?"